In [ ]:
import sys
import os
sys.path.append('/g/data/jk72/ll6859/ACCESS_aero_eval/')
from aercode import *
import matplotlib 
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import xarray as xr
import glob as gb
import cartopy.crs as ccrs
import seaborn as sns
import scipy.special
from datetime import datetime

import dask
import warnings
from dask.distributed import Client

In [ ]:
plt.style.use('default')
warnings.filterwarnings('ignore')

## Table of Contents: <a class="anchor" id="top"></a>
* [Overall Evaluation](#first-bullet)
* [Regional Evaluation](#second-bullet)
* [Temporal Evaluation](#third-bullet)
* [Radon Evaluation](#fourth-bullet)
* [BLN Evaluation](#fifth-bullet)

# Load in data

# Overall Evaluation <a class="anchor" id="first-bullet"></a>
* [Back to Top](#top)

## Regional Evaluation <a class="anchor" id="second-bullet"></a>
* [Back to Top](#top)

## Temporal Evaluation <a class="anchor" id="third-bullet"></a>
* [Back to Top](#top)

## Radon Evaluation <a class="anchor" id="fourth-bullet"></a>
* [Back to Top](#top)

## BLN Evaluation <a class="anchor" id="fifth-bullet"></a>
* [Back to Top](#top)